<a href="https://colab.research.google.com/github/blusewill/ytvideo-decipher/blob/dev/ytvideo_decipher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ytvideo-decipher

The Project that is kinda fork to the [ytvideo-whisper](https://github.com/blusewill/ytvideo-whisper)

Uses the [Decipher Project](https://github.com/dsymbol/decipher)

## Install Decipher

In [1]:
! nvidia-smi
! apt update && apt install ffmpeg fonts-open-sans
! pip install git+https://github.com/dsymbol/decipher
! pip install yt-dlp

Sun May 21 04:13:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Add Google Drive Access 
(at Google Drive/Decipher)

In [2]:
from google.colab import drive
import os

path = '/content/gdrive/MyDrive/decipher'
drive.mount('/content/gdrive')

if not os.path.exists(path):
  os.mkdir(path)

Mounted at /content/gdrive


# Download the YouTube Video using yt-dlp

# Settings

In [3]:
YouTube_Video_Link = "https://www.youtube.com/watch?v=yCTSOAy3vvg" #@param {type:"string"}
new_filename = "I Ordered Pizza From the Nintendo Wii (and you can too)" #@param {type:"string"}
# Change it to srt file type
new_filename = os.path.splitext(new_filename)[0] + ".srt"
shutdown_after_complete = "yes" #@param ['yes','no']

In [4]:
import yt_dlp

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'aac',
        'preferredquality': '192',
    }],
        'outtmpl': 'content/audio',
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([YouTube_Video_Link])

[youtube] Extracting URL: https://www.youtube.com/watch?v=yCTSOAy3vvg
[youtube] yCTSOAy3vvg: Downloading webpage
[youtube] yCTSOAy3vvg: Downloading android player API JSON
[youtube] yCTSOAy3vvg: Downloading player e50626d8
[info] yCTSOAy3vvg: Downloading 1 format(s): 251
[dashsegments] Total fragments: 1
[download] Destination: content/audio
[download] 100% of    7.82MiB in 00:00:00 at 63.30MiB/s              
[ExtractAudio] Destination: content/audio.m4a
Deleting original file content/audio (pass -k to keep)


# Transcribe the video

In [5]:
input = "content/audio.m4a"
output_dir = "/content/gdrive/MyDrive/decipher/result"
model = "medium" #@param ['tiny.en','tiny','base.en','base','small.en','small','medium.en','medium','large-v1','large-v2','large']
language = ""
task = "transcribe"
subtitle_action = "None"

from decipher.action import transcribe
import os

transcribe(
    input, 
    output_dir,
    model,
    language if language else None,
    task,
    subtitle_action if subtitle_action != "None" else None
)



Converting audio.m4a to audio.aac...


audio.m4a: 100%|██████████| 464/464 [00:00<00:00, 229121.39 seconds/s]
100%|█████████████████████████████████████| 1.42G/1.42G [00:33<00:00, 46.2MiB/s]


Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:04.320]  So here's a sentence I never thought I would say in 2023.
[00:04.320 --> 00:11.240]  I ordered this pizza from the Nintendo Wii, a console that is almost 17 years old.
[00:11.240 --> 00:13.400]  Is it the easiest way to order a pizza?
[00:13.400 --> 00:14.640]  Well, no.
[00:14.640 --> 00:17.800]  But when has this channel ever been known for doing things the easy way?
[00:17.800 --> 00:20.440]  Hello everybody and welcome back to another video.
[00:20.440 --> 00:24.360]  You know, we've done a decent amount of Wii modding stuff on the channel recently,
[00:24.360 --> 00:28.160]  but what I'm going to show you today originally didn't require any of that,
[00:28.160 --> 00:33.320]  because this was an official service made available by Nintendo in the Wii shop.
[00:33.320 --> 00:36.280]  It's called the Demaille channel, and if you've never 

# Rename the srt file

In [6]:
current_file_path = '/content/gdrive/MyDrive/decipher/result/audio.srt'
directory_path = os.path.dirname(current_file_path)
file_name = os.path.basename(current_file_path)
new_file_path = os.path.join(directory_path, new_filename)
os.rename(current_file_path, new_file_path)

# Auto Shutdown the Runtime

In [7]:
if shutdown_after_complete.lower() == "yes":
    from google.colab import runtime
    runtime.unassign()